[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/02_Safety_and_Supervisor.ipynb)


In [ ]:
import os
import sys
from pathlib import Path
import subprocess

def _ensure_repo():
    workspace = os.environ.get('GITHUB_WORKSPACE')
    if workspace:
        repo_root = Path(workspace).resolve()
        src_path = repo_root / 'src'
        if src_path.exists():
            sys.path.insert(0, str(src_path))
            return repo_root
    repo_root = Path().resolve()
    src_path = repo_root / 'src'
    if src_path.exists():
        sys.path.insert(0, str(src_path))
        return repo_root
    repo_dir = Path('IINTS-SDK')
    if not repo_dir.exists():
        subprocess.check_call(['git', 'clone', 'https://github.com/python35/IINTS-SDK.git'])
    repo_root = repo_dir.resolve()
    os.chdir(repo_root)
    src_path = repo_root / 'src'
    sys.path.insert(0, str(src_path))
    return repo_root

_ensure_repo()


# Supervisor and Safety Rules

Inspect the safety layer and see how decisions are modified.


In [ ]:
from iints.core.supervisor import IndependentSupervisor

supervisor = IndependentSupervisor(max_insulin_per_bolus=2.0, max_60min=3.0)

decision = supervisor.evaluate_safety(
    current_glucose=75,
    proposed_insulin=3.0,
    current_time=0.0,
    current_iob=1.5,
)

decision


## Safety report after a short run


In [ ]:
import json
from iints.presets import load_presets

import iints
from iints.core.algorithms.correction_bolus import CorrectionBolus

presets = load_presets()
preset = next(p for p in presets if p["name"] == "pizza_paradox")

outputs = iints.run_simulation(
    algorithm=CorrectionBolus,
    scenario=preset["scenario"],
    patient_config=preset["patient_config"],
    duration_minutes=240,
    time_step=preset["time_step_minutes"],
    seed=7,
    output_dir="results/safety_demo",
)

outputs["safety_report"]
